<a href="https://colab.research.google.com/github/chloesung/Song-Lyrics-Generator/blob/main/K-POP_%EB%85%B8%EB%9E%98%EA%B0%80%EC%82%AC_%EC%83%9D%EC%84%B1%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎤 🎧 💿 K-POP 세대별 노래 가사 생성기 💿 🎧 🎤

###  🪄 준비 과정! 코드를 모두 돌려주세요 

In [ ]:
!pip install transformers
import os
from google.colab import drive

import pandas as pd
import numpy as np
import random

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Authentication을 진행합니다
from google.colab import auth
auth.authenticate_user() 

# 필요한 파일을 다운로드 해줍니다

def folder_list(folder_id):
  from googleapiclient.discovery import build
  gdrive = build('drive', 'v3').files()
  res = gdrive.list(q="'%s' in parents" % folder_id).execute()
  return [f['id'] for f in res['files']]

'''download all files from a gdrive folder to current directory'''
def folder_download(folder_id):
  for fid in folder_list(folder_id):
    !gdown -q --id $fid

folder_id = "1aaLQZCaMlkWwqmOsU3-R-cImUkFh9BwU"
folder_download(folder_id)

In [ ]:
#모델, 토크나이저 불러오기

LOAD_MODEL_DIR = "/content"
model = GPT2LMHeadModel.from_pretrained(LOAD_MODEL_DIR)
from transformers import BertTokenizerFast, GPT2LMHeadModel
tokenizer = BertTokenizerFast.from_pretrained(LOAD_MODEL_DIR)

### 본격적인 가사 생성! 🎶 🎵

In [ ]:
#@markdown 가사와 세대를 입력해주세요:

generation = "1\uC138\uB300" #@param ['1세대', '2세대', '3세대']
lyrics = "\uBA38\uB9AC\uBD80\uD130 \uBC1C\uB05D\uAE4C\uC9C0" #@param {type:"string"}

#@markdown 생성할 가사의 개수를 고르세요:
num = 3 #@param {type:"slider", min:1, max:10, step:1}

#input 생성하기
given = '<|startoftext|>' + '<' + generation + '> ' + lyrics + '<br>'
input = tokenizer.encode(given, return_tensors='pt')
input = input[:, 1:]  # remove cls token

#output 만들기
sample_outputs = model.generate(
                                input, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=50, 
                                top_p=0.95, 
                                max_length = 300,
                                num_return_sequences=num,
                                repetition_penalty=1.3
                                )

# Print the generated lyrics
for i, sample_output in enumerate(sample_outputs):
    lyric = tokenizer.decode(sample_output, skip_special_tokens=True)
    print("\n" + ("-"*50) + "[" + str(i+1) + "]" + ("-"*50) + "\n")
    print("세대: ", generation,'\n' "입력된 가사: ", lyrics,'\n')
    print(lyric.replace('<br>', '\n'))


Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
